In [1]:
import pandas as pd

In [2]:
def dataPrep(ExpName, GE_df,  Drug_df):
    GE_df.dropna(inplace=True)

    Drug_df.rename(columns={'COSMIC_ID':'SAMPLE_ID'},inplace=True)
    #Drug_df.dropna(inplace=True)
    #Drug_df.drop(columns=['DATASET', 'NLME_RESULT_ID', 'NLME_CURVE_ID','PUTATIVE_TARGET', 'CELL_LINE_NAME', 'SANGER_MODEL_ID',
     #                    'PATHWAY_NAME', 'COMPANY_ID','WEBRELEASE', 'MIN_CONC', 'MAX_CONC','SMILES', 'WORDS', 'AUC', 'RMSE',
      #                   'Z_SCORE', 'index'],inplace=True)
    GBM_DRUG_LIST = ['Crizotinib','Docetaxel','Temozolomide', 'Camptothecin','Carmustine', 'Irinotecan','Nimotuzumab','Temozolomide']
    TARGET = 'LN_IC50'
    IRRELEVANT_FETURES = ['DATASET', 'NLME_RESULT_ID', 'NLME_CURVE_ID', 'COSMIC_ID',
       'CELL_LINE_NAME', 'SANGER_MODEL_ID', 'TCGA_DESC', 'DRUG_ID',
       'DRUG_NAME', 'PUTATIVE_TARGET', 'PATHWAY_NAME', 'COMPANY_ID',
       'WEBRELEASE', 'MIN_CONC', 'MAX_CONC', 'AUC', 'RMSE',
       'Z_SCORE', 'index', 'PUBCHEM_ID', 'SMILES', 'WORDS', 'SAMPLE_ID',]
    
    #Drug_df.dropna(inplace=True)
    
    Drug_df_GBM = Drug_df[Drug_df['TCGA_DESC']=='GBM'].copy(deep=True)

    Drug_df.drop(Drug_df_GBM.index,inplace=True)

    #GBM_known_drugs_test_df =  Drug_df_GBM[(Drug_df_GBM['DRUG_NAME'].isin(GBM_DRUG_LIST))].copy(deep=True)

    #Drug_df_GBM.drop(GBM_known_drugs_test_df.index, inplace=True)

    panCancer_train_df = pd.merge(Drug_df,GE_df, on=['SAMPLE_ID'], how='left')
    GBM_train_df = pd.merge(Drug_df_GBM,GE_df, on=['SAMPLE_ID'], how='left')
    #GBM_test_df = pd.merge(GBM_known_drugs_test_df,GE_df, on=['SAMPLE_ID'], how='left')

    Drug_df= None
    GE_df = None
    Drug_df_GBM = None
    GBM_known_drugs_test_df = None

    panCancer_train_df.to_csv(f'/data/user/rsharma3/nbotw/Project-D2GNETs/data/panCancer_train_{ExpName}_df.csv',index=False)
    panCancer_train_df=None

    GBM_train_df.dropna(inplace=True)
    GBM_train_df.to_csv(f'/data/user/rsharma3/nbotw/Project-D2GNETs/data/GBM_train_{ExpName}_df.csv',index=False)
    GBM_train_df = None

    #GBM_test_df.dropna(inplace=True)
    #GBM_test_df.to_csv(f'/data/user/rsharma3/nbotw/Project-D2GNETs/data/GBM_test_{ExpName}_df.csv',index=False)
    #GBM_test_df = None

# ML modeling Data preparation using entire gene set

### Data Praparation without Dimensions reduction

In [8]:
GE_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/AllGeneExpression.csv')
Drug_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/GDSC_Drug_Feature_Engineered_new.csv')

/scratch/local/ipykernel_15354/2804455200.py:2: DtypeWarning: Columns (9,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  Drug_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/GDSC_Drug_Feature_Engineered_new.csv')


In [9]:
dataPrep('All_GE_new_drug_features', GE_df, Drug_df)

### Data Preparation with Dimension reduced data (through RANs)

In [10]:
Drug_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/GDSC_Drug_Feature_Engineered_with_RANs_new_drug_FE.csv')
GE_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/GE_dimension_reduced_RANS.csv')

In [11]:
dataPrep('DRwRANS', GE_df, Drug_df)

# ML modeling Data Preparation using selected genes

### Selection based on cancer gene censue 657 genes

In [18]:
GE_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/AllGeneExpression.csv')
Drug_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/GDSC_Drug_Feature_Engineered_new_scaled.csv')
censusGN = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/Census_allFri_May_6_15_00_18_2022.csv')


In [19]:
GE_df.max().max()

1674021.0

In [20]:
X_cols = list(set(GE_df.columns).intersection(set(censusGN['Gene Symbol'].values)))
GE_df = GE_df[['SAMPLE_ID']+X_cols]

In [21]:
dataPrep('CGC_657_new_drug_features_v2', GE_df, Drug_df)

In [22]:
coldf = pd.DataFrame(columns=X_cols)
coldf.to_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/657_Gene_name.csv',index=False)

# ML modeling Data Preparation with CGC's 657 genes and Dimension Reduced drug features

In [12]:
GE_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/AllGeneExpression.csv')
censusGN = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/Census_allFri_May_6_15_00_18_2022.csv')
X_cols = list(set(GE_df.columns).intersection(set(censusGN['Gene Symbol'].values)))
GE_df = GE_df[['SAMPLE_ID']+X_cols]
Drug_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/GDSC_Drug_Feature_Engineered_with_RANs_new_drug_FE.csv')



In [13]:
dataPrep('CGC_657_DR_Drug_features_new', GE_df, Drug_df)

In [18]:
panCancer_train_df = pd.read_csv('/data/user/rsharma3/nbotw/Project-D2GNETs/data/panCancer_train_CGC_657_new_drug_features_v2_df.csv')


In [32]:
24+203-6

221

In [36]:
#panCancer_train_df.columns[24:221].to_list()
panCancer_train_df.columns[221:].to_list()

['SBDS',
 'FGFR1',
 'DCC',
 'N4BP2',
 'MSH6',
 'ARHGEF12',
 'NCKIPSD',
 'HRAS',
 'CASP3',
 'CDK6',
 'POT1',
 'GATA2',
 'NFIB',
 'PIK3CB',
 'RHOH',
 'MYB',
 'SIX1',
 'TGFBR2',
 'BCL11A',
 'TERT',
 'ACSL6',
 'DDB2',
 'NRAS',
 'FGFR3',
 'GRIN2A',
 'HLA-A',
 'TMEM127',
 'NTRK3',
 'SRC',
 'ZNF479',
 'LSM14A',
 'CDH10',
 'VTI1A',
 'TLX3',
 'CD28',
 'CTNND2',
 'TEC',
 'SIX2',
 'GATA3',
 'HOXA13',
 'FLI1',
 'PRDM1',
 'SDC4',
 'FLNA',
 'MUTYH',
 'KIT',
 'CIITA',
 'BIRC6',
 'ARHGEF10L',
 'PML',
 'BRCA1',
 'SMAD3',
 'CRTC1',
 'SRSF2',
 'FUS',
 'CANT1',
 'MAF',
 'HOOK3',
 'HIP1',
 'GOLGA5',
 'DDIT3',
 'CCND3',
 'SGK1',
 'ARID1A',
 'IKBKB',
 'MAPK1',
 'PAFAH1B2',
 'CBLC',
 'CHCHD7',
 'EZH2',
 'STRN',
 'FHIT',
 'MLH1',
 'MDM2',
 'BCL6',
 'RECQL4',
 'CASP8',
 'TET2',
 'LIFR',
 'EML4',
 'MITF',
 'GPHN',
 'HIST1H3B',
 'AFF4',
 'CPEB3',
 'ASXL2',
 'DDX5',
 'ALDH2',
 'GOPC',
 'SS18L1',
 'TFPT',
 'HOXC13',
 'CD74',
 'CARS',
 'RAP1GDS1',
 'NACA',
 'RET',
 'POLD1',
 'RBM10',
 'TOP1',
 'CARD11',
 'SUFU',
 'S